dataset from: https://www.kaggle.com/datasets/davidbroberts/piano-triads-wavset

In [24]:
import os
import numpy as np
import pandas as pd
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPooling1D
import seaborn as sns
import matplotlib.pyplot as plt

In [25]:
data_path = "/Users/elaineran/Downloads/archive2/piano_triads/"
metadata_path = "/Users/elaineran/Desktop/summer-project/triads_modified.csv"

In [26]:
import chord_edit
import random

funcs = [chord_edit.time_stretch, chord_edit.add_noise, chord_edit.time_stretch_and_add_noise]
i = 1
while i <11:
    rand = random.randint(0, 2)
    funcs[rand](data_path, metadata_path, i)
    i+=1

In [27]:
def load_data(data_path, metadata_path):
    features = []
    labels = []


    metadata = pd.read_csv(metadata_path)


    for index, row in metadata.iterrows():
        file_path = os.path.join(data_path, f"{row['Chord']}.wav")


        # Load the audio file and resample it
        target_sr = 22050
        audio, sample_rate = librosa.load(file_path, sr=target_sr)

        # Check if the audio length is less than the default n_fft size
        if len(audio) < 2048:
            # Pad the audio file with zeros
            audio = np.pad(audio, (0, 2048 - len(audio)), mode='constant')



        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=audio, sr=target_sr, n_mfcc=40)
        mfccs_scaled = np.mean(mfccs.T, axis=0)


        # Append features and labels
        features.append(mfccs_scaled)
        labels.append(row['Chord_modified'])


    return np.array(features), np.array(labels)



In [28]:
features, labels = load_data(data_path, metadata_path)


# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_onehot = to_categorical(labels_encoded)


In [29]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_onehot, test_size=0.2, random_state=42)


In [30]:
input_shape = (X_train.shape[1], 1)
model = Sequential()
model.add(Conv1D(64, 3, padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv1D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(le.classes_), activation='softmax'))


In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [33]:
def make_predictions(model, le, file_path):
    audio, sample_rate = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    features = mfccs_scaled.reshape(1, mfccs_scaled.shape[0], 1)
    predicted_vector = model.predict(features)
    predicted_class_index = np.argmax(predicted_vector, axis=-1)
    return le.inverse_transform(predicted_class_index)[0]


In [34]:
# Save the initial weights
initial_weights = model.get_weights()


# Dictionary to store old predictions
old_predictions = {}
new_predictions = {}


# List of test files and their true labels
#test_files = [
    #("/Users/elaineran/Downloads/archive/fold1/101415-3-0-2.wav", "Dog bark"),
    #("/Users/elaineran/Downloads/archive/fold1/101415-3-0-3.wav", "Dog bark"),
    #("/Users/elaineran/Downloads/archive/fold1/102305-6-0-0.wav", "Gun shots"),
    #("/Users/elaineran/Downloads/archive/fold1/103074-7-0-2.wav", "Jack hammer"),
    #("/Users/elaineran/Downloads/archive/fold1/103074-7-4-3.wav", "Jack hammer")
#]


# Make predictions before training
#for file_path, true_label in test_files:
    #predicted_label_before = make_predictions(model, le, file_path)
    #old_predictions[file_path] = predicted_label_before


In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Train on 3168 samples, validate on 792 samples
Epoch 1/100
3168/3168 [==============================] - 5s 2ms/step - loss: 5.6763 - accuracy: 0.0319 - val_loss: 3.5490 - val_accuracy: 0.0707
Epoch 2/100
3168/3168 [==============================] - 4s 1ms/step - loss: 3.5616 - accuracy: 0.0496 - val_loss: 3.4005 - val_accuracy: 0.0821
Epoch 3/100
3168/3168 [==============================] - 5s 1ms/step - loss: 3.3373 - accuracy: 0.0922 - val_loss: 2.8614 - val_accuracy: 0.2247
Epoch 4/100
3168/3168 [==============================] - 4s 1ms/step - loss: 2.8264 - accuracy: 0.2030 - val_loss: 2.1813 - val_accuracy: 0.4192
Epoch 5/100
3168/3168 [==============================] - 4s 1ms/step - loss: 2.3655 - accuracy: 0.3242 - val_loss: 1.7457 - val_accuracy: 0.5556
Epoch 6/100
3168/3168 [==============================] - 5s 1ms/step - loss: 1.9485 - accuracy: 0.4410 - val_loss: 1.3811 - val_accuracy: 0.6427
Epoch 7/100
3168/3168 [==============================] - 5s 2ms/step - loss: 1.6039

In [ ]:
# plotting from history

loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = list(range(len(loss)))

figsize = (6, 4)
fig, axis1 = plt.subplots(figsize=figsize)
plot1_lacc = axis1.plot(epochs, acc, 'navy', label='accuracy')
plot1_val_lacc = axis1.plot(epochs, val_acc, 'deepskyblue', label="validation accuracy")

plot1_loss = axis1.plot(epochs, loss, 'red', label='loss')
plot1_val_loss = axis1.plot(epochs, val_loss, 'lightsalmon', label="validation loss")


plots = plot1_loss + plot1_val_loss
labs = [plot.get_label() for plot in plots]
axis1.set_xlabel('Epoch')
axis1.set_ylabel('Loss/Accuracy')
plt.title("Loss/Accuracy History")
plt.tight_layout()
axis1.legend(loc='lower right')
plt.savefig("/Users/elaineran/Desktop/summer-project/figures/Loss-Accuracy")
plt.show()

In [ ]:
test_predictions = model.predict(X_test)

In [ ]:
predicted_class_indices = np.argmax(test_predictions, axis=1)
predicted_classes = le.inverse_transform(predicted_class_indices)
print(predicted_classes)

In [ ]:
y_true = np.argmax(y_test, axis=1)
true_classes = le.inverse_transform(y_true)
print(true_classes)

In [ ]:
unique_labels = np.unique(np.concatenate([y_true, predicted_class_indices]))


# Calculate accuracy
accuracy = accuracy_score(y_true, predicted_class_indices)
print("Accuracy:", accuracy)


# Generate a classification report
print(classification_report(y_true, predicted_class_indices, labels=unique_labels, target_names=le.inverse_transform(unique_labels)))


# Confusion matrix
cm = confusion_matrix(y_true, predicted_class_indices)
print("Confusion Matrix:\n", cm)

In [ ]:


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.savefig("/Users/elaineran/Desktop/summer-project/figures/Confusion-Matrix")
plt.show()
